In [31]:
import torch 

# Load the saved dataloader and logits
model_name = 'ViT-B-32-quickgelu'
model_name = 'ViT-H-14-378-quickgelu'
logits = torch.load(f'total_logits_{model_name}.pt')
device = logits.device
targets = torch.load(f'total_targets_{model_name}.pt')

print(logits.shape)
print(targets.shape)

torch.Size([50000, 1000])
torch.Size([50000])


In [ ]:

# Compute the accuracy

with torch.no_grad(): # No need to track gradients for accuracy calculation
    # Get the index of the max logit score (predicted class)
    _, predicted_classes = torch.max(logits, dim=1)

    # Compare predicted classes with the true targets
    correct_predictions = (predicted_classes == targets)

    # Calculate the mean accuracy
    top1_accuracy = correct_predictions.float().mean()
    print(f"\nTop-1 Accuracy: {top1_accuracy.item():.4f}")



Top-1 Accuracy: 0.8436


In [ ]:
import torch.nn as nn 

lr = 1
max_iter = 100000
log_iter = 1000
loss_his = torch.zeros(max_iter)
max_his = torch.zeros(max_iter//log_iter) 
acc_his = torch.zeros(max_iter//log_iter)


# Initialize the parameter outside the loop to avoid reinitialization in each iteration
param = nn.Linear(logits.shape[1], 1).to(device)
loss_fn = nn.CrossEntropyLoss()

for iter in range(max_iter):
    # Compute the output
    output = loss_fn(logits + param.weight, targets)
    
    # Compute gradients
    param.zero_grad()  # Clear previous gradients
    output.backward()  # Backpropagation

    loss_his[iter] = output.item()
    # Update parameters
    with torch.no_grad():
        param.weight -= lr * param.weight.grad
        param.weight -= torch.min(param.weight)  # Ensure non-negative weights
    
    if iter % log_iter == 0:
        # top 1-acc
        _, pred = torch.max(logits + param.weight, 1)
        acc = (pred == targets).float().mean()
        max_his[iter//log_iter] = param.weight.max().item()
        acc_his[iter//log_iter] = acc.item()
        print(f"Iteration {iter}: Loss = {output.item()}, std = {param.weight.std().item()}, min = {param.weight.min().item()}, max = {param.weight.max().item()}, acc = {acc.item()}")





Iteration 0: Loss = 0.6666797399520874, std = 0.0184860210865736, min = 0.0, max = 0.0633256733417511, acc = 0.8437199592590332
Iteration 1000: Loss = 0.636522650718689, std = 0.1741292029619217, min = 0.0, max = 1.6848067045211792, acc = 0.8478800058364868
Iteration 2000: Loss = 0.6164097785949707, std = 0.31445828080177307, min = 0.0, max = 3.0328052043914795, acc = 0.8502799868583679
Iteration 3000: Loss = 0.6022956371307373, std = 0.43126922845840454, min = 0.0, max = 4.116040229797363, acc = 0.8511599898338318
Iteration 4000: Loss = 0.5920002460479736, std = 0.5300028324127197, min = 0.0, max = 5.026841640472412, acc = 0.8521999716758728
Iteration 5000: Loss = 0.5842309594154358, std = 0.6146802306175232, min = 0.0, max = 5.777279853820801, acc = 0.852620005607605
Iteration 6000: Loss = 0.5781932473182678, std = 0.6882807016372681, min = 0.0, max = 6.4720234870910645, acc = 0.8532599806785583
Iteration 7000: Loss = 0.5733891129493713, std = 0.7529970407485962, min = 0.0, max = 7.0

In [17]:
print("Final weights:", param.weight)
print("mean logits", logits.mean())

Final weights: Parameter containing:
tensor([[2.3996, 3.1800, 2.9355, 0.0000, 2.1533, 2.5203, 2.4460, 2.4669, 3.9907,
         2.6469, 1.9435, 1.1919, 5.5982, 3.4883, 2.6283, 1.8619, 6.5320, 1.4669,
         1.6356, 2.7517]], device='cuda:0', requires_grad=True)
mean logits tensor(9.1875, device='cuda:0', dtype=torch.float16)
